Aims:
  - Reproduce results in Sánchez et al. (2017)
  - This is to understand how to deal with exchange fluxes in the context of media compositions, and understand computing growth rate, reactions that flow into the objective function.

Import dependencies

In [ ]:
%reload_ext autoreload
%autoreload 1
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import cobra
import escher

# Load model

The one used in the paper

In [ ]:
# Enzyme-constrained Yeast 7, batch
# In this model, the total amount of enzyme is limited, rather than each enzyme being limited separately.
# This assumes an average in vivo saturation of enzymes (sigma) of 0.44,
# value estimated by fitting the model to growth on glucose,
# and the total protein content (P_{total}) of 0.5 g/gDW, assumed to be constant.
model = cobra.io.read_sbml_model("./models/ecYeast7_v1.0_batch.xml")
model

More recent alternatives

In [ ]:
model = cobra.io.read_sbml_model("./models/ecYeastGEM_batch.xml")
model

In [ ]:
model = cobra.io.read_sbml_model("./models/ecYeastGEM.xml")
model

In [ ]:
model = cobra.io.load_yaml_model("./models/ecYeastGEMfull_GECKO.yml")
model

# Set exchange bounds

## Mimimal media

Glucose exchange

In [ ]:
model.reactions.get_by_id('r_1714')

Initially, the glucose exchange bounds are (0,0), meaning there is no glucose exchange.  To simulate glucose uptake, we need to unconstrain the bound, which means setting the bounds to (-1000,0).

In [ ]:
model.reactions.r_1714.bounds = (-1000, 0)

Ethanol exchange -- an example that would yield a very different value.  Note that stoichiometry goes in the opposite direction as glucose.

In [ ]:
model.reactions.get_by_id('r_1761')

Set bounds

In [ ]:
model.reactions.r_1714.bounds = (0, 0)
model.reactions.r_1761.bounds = (-1000, 0)
#model.reactions.r_1761_REV.bounds = (0, 1)

Glycerol exchange

In [ ]:
model.reactions.get_by_id('r_1808')

In [ ]:
model.reactions.r_1714.bounds = (0, 0)
model.reactions.r_1808.bounds = (-1000, 0)
model.reactions.r_1808_REV.bounds = (0, 1)

## Minimal media + amino acids

In [ ]:
amino_exch_list = [
    'r_1873', 'r_1879', 'r_1880', 'r_1881', 'r_1883',
    'r_1889', 'r_1891', 'r_1810', 'r_1893', 'r_1897',
    'r_1899', 'r_1900', 'r_1902', 'r_1903', 'r_1904',
    'r_1906', 'r_1911', 'r_1912', 'r_1913', 'r_1914',
]

for amino in amino_exch_list:
    model.reactions.get_by_id(amino).bounds = (-1000, 0)

In [ ]:
# glucose
model.reactions.r_1714.bounds = (-1000, 0)

# Optimise model

In [ ]:
# Biomass pseudoreaction, the objective function
model.reactions.get_by_id('r_4041')

In [ ]:
model.reactions.get_by_id('r_4041').bounds = (0.0,1000)

In [ ]:
solution = model.optimize()

biomass = model.reactions.get_by_id('r_4041')
print(f'Predicted μmax is {biomass.flux:.4f} h-1')

If we simulate glucose uptake in minimal media, the result should be 0.3790 $h^{-1}$, which agrees with figure 4B (dark indigo circle).

Glycerol should give 0.2503, which agreed with the figure too.

For some reason ethanol doesn't agree with the figure.

Glucose uptake in minimal media plus amino acids yields 0.5137, agreeing with the figure (dark indigo square)

# Gene deletions

Start over

In [ ]:
model = cobra.io.read_sbml_model("./models/ecYeast7_v1.0_batch.xml")

Unrestrict glucose exchange

In [ ]:
model.reactions.r_1714.bounds = (-1000, 0)

Focus on these reactions: O<sub>2</sub> consumption and CO<sub>2</sub> production

In [ ]:
# O2
model.reactions.get_by_id('r_1992')

In [ ]:
# CO2
model.reactions.get_by_id('r_1672')

Optimise wild-type model

In [ ]:
solution = model.optimize()
model.summary()

In [ ]:
print(f'growth: {solution["r_4041"]}')
print(f'oxygen: {solution["r_1992_REV"]}')
print(f'carbon dioxide: {solution["r_1672"]}')

> This (probably) makes sense as in figure 6A, when the dilution rate is 0.38 (off the chart), the oxgen consumption is about 2 and the carbon dioxide production is > 20.

Set dilution rate, i.e. force growth rate.

In [ ]:
model.reactions.r_4041.bounds = (0.15, 0.15)
solution = model.optimize()
print(f'growth: {solution["r_4041"]}')
print(f'oxygen: {solution["r_1992_REV"]}')
print(f'carbon dioxide: {solution["r_1672"]}')

Sweep

**BUG: Running the code the 1st time gives a different result from the 2nd-later times.  The first time aligns the most to fig 6A, but have weird kinks in them.  The kinks could be floating-point rounding issues propagating to the optimisations, but I have no idea why outputs are different each time.  I suspect this may be a Python-specific problem that could be solved by using Julia.**

In [ ]:
model = cobra.io.read_sbml_model("./models/ecYeast7_v1.0_batch.xml")
model.reactions.r_1714.bounds = (-1000, 0)

In [ ]:
dil_rates = np.linspace(0.1, 0.35, 600)
o2_flux = []
co2_flux = []

for dil_rate in dil_rates:
    model.reactions.r_4041.bounds = (dil_rate, dil_rate)
    solution = model.optimize()
    o2_flux.append(solution.fluxes["r_1992_REV"])
    co2_flux.append(solution.fluxes["r_1672"])

In [ ]:
fig, ax = plt.subplots()
ax.plot(dil_rates, o2_flux, 'b')
ax.plot(dil_rates, co2_flux, 'r')

Delete NDI1, NDE1, NDE2

In [ ]:
model = cobra.io.read_sbml_model("./models/ecYeast7_v1.0_batch.xml")
print('model loaded')
# This model has no GPR, so need to manually identify reactions to knock out
reactions_to_ko = [
    'r_0773No1',
    'draw_prot_P32340',
    'r_0770No2',
    'draw_prot_P40215',
    'r_0770No1',
    'draw_prot_Q07500',
]
for reaction in reactions_to_ko:
    model.reactions.get_by_id(reaction).knock_out()

model.reactions.r_1714.bounds = (-1000, 0)
print('bounds set')

dil_rates = np.linspace(0.1, 0.35, 600)
o2_flux = []
co2_flux = []

for dil_rate in dil_rates:
    model.reactions.r_4041.bounds = (dil_rate, dil_rate)
    solution = model.optimize()
    o2_flux.append(solution.fluxes["r_1992_REV"])
    co2_flux.append(solution.fluxes["r_1672"])
print('optimisations done')
    
fig, ax = plt.subplots()
ax.plot(dil_rates, o2_flux, 'b')
ax.plot(dil_rates, co2_flux, 'r')

> Despite the bug, it's clear that the point where oxygen consumption and carbon dioxide production diverge shift from 0.3ish to 0.2ish, broadly agreeing with figure 6A in Sánchez et al. (2017).  So, merely blocking the reactions should make sense.